# 1. Load packages

In [1]:
from analysis_tools.common import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


# 2. Load dataset

In [2]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

with ProgressBar():
    X_train_full = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.train)])
    X_test       = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.test)])
y_train_full = train_full_data_meta[['label']]
    
print("- Number of train full data:", len(X_train_full))
print("- Number of test data:", len(X_test))

[########################################] | 100% Completed |  7.5s
[########################################] | 100% Completed |  3.6s
- Number of train full data: 4277
- Number of test data: 2154


# 3. Training

In [3]:
from sklearn.preprocessing import OneHotEncoder

IMG_SIZE    = 700
input_shape = (IMG_SIZE, IMG_SIZE, 3)

with ProgressBar():
    X_train_full = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_train_full]))
    X_test       = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_test]))
y_enc = OneHotEncoder(sparse=False, dtype=bool)
y_train_full = y_enc.fit_transform(y_train_full)
n_classes = len(y_enc.categories_[0])
print("- Number of classes:", n_classes)

[########################################] | 100% Completed |  3.6s
[########################################] | 100% Completed |  1.2s
- Number of classes: 88


In [4]:
aug_model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def preprocess(ds, training, batch_size, augment=True):
    ds = ds.cache().batch(batch_size)
    if training:
        ds = ds.shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)
        if augment:
            ds = ds.map(lambda X, y, sw: (aug_model(X), y, sw), num_parallel_calls=tf.data.AUTOTUNE)
    return ds

# fig, axes = plt.subplots(5, 15, figsize=(30, 10))
# for row, ax_cols in enumerate(axes):
#     for ax in ax_cols:
#         ax.imshow(aug_model(X_train_full[row]))
#         ax.axis('off')
# plt.show()

## 3.1 Fix pretrained model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, stratify=y_train_full)
sample_weight_train = compute_sample_weight(class_weight='balanced', y=y_train.argmax(1))
sample_weight_val   = compute_sample_weight(class_weight='balanced', y=y_val.argmax(1))

train_ds = preprocess(tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weight_train)), True, BATCH_SIZE)
val_ds   = preprocess(tf.data.Dataset.from_tensor_slices((X_val, y_val, sample_weight_val)), False, BATCH_SIZE)
test_ds  = preprocess(tf.data.Dataset.from_tensor_slices(X_test), False, BATCH_SIZE)

print("- train.shape:", X_train.shape, y_train.shape)
print("- val.shape:", X_val.shape, y_val.shape)
print("- test.shape:", X_test.shape)

- train.shape: (3207, 700, 700, 3) (3207, 88)
- val.shape: (1070, 700, 700, 3) (1070, 88)
- test.shape: (2154, 700, 700, 3)


In [6]:
from tensorflow_addons.metrics import F1Score

def build_model(n_classes, strategy):
    with strategy.scope():
        base_model = keras.applications.EfficientNetB6(include_top=False, input_shape=input_shape)
        base_model.trainable = False

        inputs  = keras.Input(input_shape)
        hidden  = base_model(inputs, training=False)
        hidden  = keras.layers.GlobalAveragePooling2D()(hidden)
        outputs = keras.layers.Dense(n_classes, activation='softmax')(hidden)
        model   = keras.Model(inputs, outputs)
        
        model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
    return model, base_model

In [7]:
from analysis_tools.modeling import *

model, base_model = build_model(n_classes, strategy)
history = model.fit(train_ds, validation_data=val_ds, epochs=1000, callbacks=get_callbacks(patience=30, plot_path=join(PATH.result, 'proposed3', 'fix_pretrained_model_nadam_b7')))

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

Epoch 46/1000
101/101 [==============================] - 65s 578ms/step - loss: 0.3483 - f1_score: 0.8002 - val_loss: 1.0678 - val_f1_score: 0.5872
Epoch 47/1000
101/101 [==============================] - 64s 576ms/step - loss: 0.3469 - f1_score: 0.8077 - val_loss: 1.1100 - val_f1_score: 0.5651
Epoch 48/1000
101/101 [==============================] - 65s 578ms/step - loss: 0.3334 - f1_score: 0.7981 - val_loss: 1.1555 - val_f1_score: 0.5633
Epoch 49/1000
101/101 [==============================] - 64s 571ms/step - loss: 0.3307 - f1_score: 0.7944 - val_loss: 1.0713 - val_f1_score: 0.5700
Epoch 50/1000
101/101 [==============================] - 66s 593ms/step - loss: 0.3119 - f1_score: 0.8090 - val_loss: 1.1002 - val_f1_score: 0.5841
Epoch 51/1000
101/101 [==============================] - 65s 578ms/step - loss: 0.3181 - f1_score: 0.8073 - val_loss: 1.0997 - val_f1_score: 0.5799
Epoch 52/1000
101/101 [==============================] - 65s 580ms/step - loss: 0.3461 - f1_score: 0.8033 - val_

## 3.2 Fine-tuning

In [8]:
with strategy.scope():
    base_model.trainable = True
    model.compile(optimizer=keras.optimizers.Nadam(2e-4), loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
history = model.fit(train_ds, validation_data=val_ds, epochs=1000, callbacks=get_callbacks(patience=30, plot_path=join(PATH.result, 'proposed3', 'fine_tuning_nadam_b7')))

Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 711 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 711 all-reduces with algorithm = nccl, num_packs = 1


ResourceExhaustedError: 4 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[11,32,350,350] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/efficientnetb7/block1c_bn/FusedBatchNormV3 (defined at /threading.py:932) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[div_no_nan_1/ReadVariableOp_2/_1178]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) Resource exhausted:  OOM when allocating tensor with shape[11,32,350,350] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/efficientnetb7/block1c_bn/FusedBatchNormV3 (defined at /threading.

# 4. Evaluation

In [ ]:
submission_file_path = join(PATH.output, 'proposed3_fine_tuning_nadam_b6.csv')

pred_test = model.predict(test_ds)
submission = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission['label'] = y_enc.inverse_transform(pred_test)
submission.to_csv(submission_file_path)
submission

# 5. Submission

In [ ]:
from dacon_submit_api.dacon_submit_api import post_submission_file

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894', 
    '어스름한 금요일 밤에', 
    'proposed3_fine_tuning_nadam_b7'
)